In [35]:
import acquire
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import re
from time import strftime
from time import sleep
import warnings
warnings.filterwarnings("ignore")

In [36]:
def get_url_list():
    '''
    Gets a list of urls from the github star ranking page, iterating through the different pages and grabbing,
    each url from the different github repositories.
    '''
    # Created an empty list that will hold all of the github links from each page.
    link_list = []
    # Created a for loop in order to get different pages for the p = {i}
    for i in range(1,99):
        # When 10 pages are accessed by the web scraper this is here to take a minute break to avoid,
        # having github throtal the webscrape.
        if i%10 == 0:
            # prints a different link in the link list to show that the pulls are working correctly
            print(link_list[i*2])
            # Prints the length of the link list to show that the list is growing correctly
            print(len(link_list))
            # shows that the sleep function is about to start and will pause function for 60 seconds
            print('waiting')
            sleep(60)
            # shows that the sleep is over
            print('continue')
            # Creates a response basically pulling in https://github.com/search?p=&q=stars%3A%3E0&s=stars&type=Repositories this link 99 times
            response = requests.get(f'https://github.com/search?p=100{i}&q=stars%3A%3E0&s=stars&type=Repositories',headers = {'user-agent': 'Codeup DS Germain'})
            # gets the text version to prepare it for beautiful soup
            html = response.text
            # use beautiful soup on the html
            soup = BeautifulSoup(html)
            # pulls the target html which holds the link of each github on that page
            text_with_link = soup.select('.v-align-middle')
            # iterates through each peice of html and will check if the regex below matches with anything.
            for link in text_with_link:
                # turning the html to a string
                link_ispy = str(link)
                # setting all the links to match
                match = re.search(r'(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-]\w+)', link_ispy)
                # checks if match is null and only passes through non null values
                if match is not None:
                    # appends the link to the link list
                    link_list.append(match.group(0))
        else:
            # Creates a response basically pulling in https://github.com/search?p=&q=stars%3A%3E0&s=stars&type=Repositories this link 99 times
            response = requests.get(f'https://github.com/search?p={i}&q=stars%3A%3E0&s=stars&type=Repositories',headers = {'user-agent': 'Codeup DS Germain'})
            # gets the text version to prepare it for beautiful soup
            html = response.text
            # use beautiful soup on the html
            soup = BeautifulSoup(html)
            # pulls the target html which holds the link of each github on that page
            text_with_link = soup.select('.v-align-middle')
            # iterates through each peice of html and will check if the regex below matches with anything.
            for link in text_with_link:
                # turning the html to a string
                link_ispy = str(link)
                # setting all the links to match
                match = re.search(r'(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-]\w+)', link_ispy)
                # checks if match is null and only passes through non null values
                if match is not None:
                    link_list.append(match.group(0))
    
    return link_list

In [37]:
repos = get_url_list()

https://github.com/pallets/flask
30
waiting
continue
https://github.com/papers-we-love/papers-we-love
110
waiting
continue
https://github.com/sindresorhus/awesome-nodejs
190
waiting
continue
https://github.com/serverless/serverless
270
waiting
continue
https://github.com/python/cpython
350
waiting
continue
https://github.com/hoppscotch/hoppscotch
430
waiting
continue
https://github.com/houshanren/hangzhou_house_knowledge
510
waiting
continue
https://github.com/rethinkdb/rethinkdb
590
waiting
continue
https://github.com/istio/istio
670
waiting
continue


In [38]:
REPOS = list(set(repos))

In [39]:
len(REPOS)

560

In [40]:
REPOS

['https://github.com/thedaviddias/Front-End-Checklist',
 'https://github.com/onevcat/Kingfisher',
 'https://github.com/FallibleInc/security-guide-for-developers',
 'https://github.com/tailwindlabs/tailwindcss',
 'https://github.com/codepath/android_guides',
 'https://github.com/github/fetch',
 'https://github.com/formulahendry/955.WLB',
 'https://github.com/ianstormtaylor/slate',
 'https://github.com/Kong/insomnia',
 'https://github.com/CymChad/BaseRecyclerViewAdapterHelper',
 'https://github.com/ggreer/the_silver_searcher',
 'https://github.com/ReactiveX/RxSwift',
 'https://github.com/SheetJS/sheetjs',
 'https://github.com/elastic/elasticsearch',
 'https://github.com/DrKLO/Telegram',
 'https://github.com/angular/material',
 'https://github.com/t4t5/sweetalert',
 'https://github.com/1c7/chinese-independent-developer',
 'https://github.com/bayandin/awesome-awesomeness',
 'https://github.com/tauri-apps/tauri',
 'https://github.com/pypa/pipenv',
 'https://github.com/judasn/IntelliJ-IDEA-T

In [41]:
r = 'https://github.com/HelloZeroNet/ZeroNet'

In [60]:
repositories = []
for repo in REPOS:
    repositories.append(repo[19:])

In [61]:
repositories

['thedaviddias/Front-End-Checklist',
 'onevcat/Kingfisher',
 'FallibleInc/security-guide-for-developers',
 'tailwindlabs/tailwindcss',
 'codepath/android_guides',
 'github/fetch',
 'formulahendry/955.WLB',
 'ianstormtaylor/slate',
 'Kong/insomnia',
 'CymChad/BaseRecyclerViewAdapterHelper',
 'ggreer/the_silver_searcher',
 'ReactiveX/RxSwift',
 'SheetJS/sheetjs',
 'elastic/elasticsearch',
 'DrKLO/Telegram',
 'angular/material',
 't4t5/sweetalert',
 '1c7/chinese-independent-developer',
 'bayandin/awesome-awesomeness',
 'tauri-apps/tauri',
 'pypa/pipenv',
 'judasn/IntelliJ-IDEA-Tutorial',
 'valyala/fasthttp',
 'pjreddie/darknet',
 'Homebrew/homebrew-cask',
 'ReactiveCocoa/ReactiveCocoa',
 'eriklindernoren/ML-From-Scratch',
 'komeiji-satori/Dress',
 'carbon-app/carbon',
 'bregman-arie/devops-exercises',
 'hwdsl2/setup-ipsec-vpn',
 'taichi-dev/taichi',
 'michalsnik/aos',
 'dotnet/corefx',
 'LingCoder/OnJava8',
 'elsewhencode/project-guidelines',
 'keon/algorithms',
 'zhaoolee/ChromeAppHeroes

In [69]:
def clean_repos(REPOS):
    REPOS = list(set(REPOS))
    repositories = []
    for repo in REPOS:
        repositories.append(repo[19:])
    return repositories

In [70]:
len(REPOS)

560

In [71]:
repositories = clean_repos(REPOS)

In [72]:
len(repositories)

560

In [73]:
repositories

['thedaviddias/Front-End-Checklist',
 'onevcat/Kingfisher',
 'FallibleInc/security-guide-for-developers',
 'tailwindlabs/tailwindcss',
 'codepath/android_guides',
 'github/fetch',
 'formulahendry/955.WLB',
 'ianstormtaylor/slate',
 'Kong/insomnia',
 'CymChad/BaseRecyclerViewAdapterHelper',
 'ggreer/the_silver_searcher',
 'ReactiveX/RxSwift',
 'SheetJS/sheetjs',
 'elastic/elasticsearch',
 'DrKLO/Telegram',
 'angular/material',
 't4t5/sweetalert',
 '1c7/chinese-independent-developer',
 'bayandin/awesome-awesomeness',
 'tauri-apps/tauri',
 'pypa/pipenv',
 'judasn/IntelliJ-IDEA-Tutorial',
 'valyala/fasthttp',
 'pjreddie/darknet',
 'Homebrew/homebrew-cask',
 'ReactiveCocoa/ReactiveCocoa',
 'eriklindernoren/ML-From-Scratch',
 'komeiji-satori/Dress',
 'carbon-app/carbon',
 'bregman-arie/devops-exercises',
 'hwdsl2/setup-ipsec-vpn',
 'taichi-dev/taichi',
 'michalsnik/aos',
 'dotnet/corefx',
 'LingCoder/OnJava8',
 'elsewhencode/project-guidelines',
 'keon/algorithms',
 'zhaoolee/ChromeAppHeroes

In [68]:
repositories

['thedaviddias/Front-End-Checklist']

In [75]:
df = pd.DataFrame([acquire.github_api_request(url) for url in REPOS])

JSONDecodeError: Expecting value: line 7 column 1 (char 6)